In [13]:
from openai import OpenAI
import pandas as pd
import json

# Load prompts & parameters

In [14]:
filename = 'prompts_user_study.csv'
#filename = 'NLEs_user_study.csv'    # Uncomment if you run the file twice and want to generate NLEs with and without LIME-rationales.
prompts = pd.read_csv(filename)

# Read the JSON file and parse it into a Python dictionary
with open('params_LLM.json', 'r') as file:
    params = json.load(file)

# Select model and setup client

In [15]:
key = params['LLM']['key']
client = OpenAI(api_key=key)

model = params['LLM']['model']

In [16]:
# instruction = "You are a helpful assistant."
instruction_rationale = """Your task is to explain the output of a Support Vector Machine for a user. The task is a text classification task.\n
            You will be provided with the input to the model, the model output, confidence_score, and LIME rationales for the model output.\n\n
            
            The input is denoted as ”Input:” and is a string of text.\n
            The model prediction is denoted as ”Output:” and can be one out of three classes [politics, leisure, science].\n
            The confidence_score is denoted as ”Confidence:” and is a value between 0 and 1. A value closer to 1 indicates that the model is more confident in its prediction.\n

            The LIME rationales are extracted by a LIME model. It provides the top-k most influential input features for the model decision in a ranked order. The first feature is the most influential.\n
            The format of a LIME rationale looks like this [feature: value].\n
            If a feature value is positive, it is rational for the predicted class. If the feature value is negative (starts with ”-”), it is a rationale against the predicted class.\n\n
            
            A LIME rationale for the output is denoted as ”Output rationale:” and looks like: [motorcycles: 0.358, BMW: 0.309, used: -0.144, known: -0.131, twin: 0.119, non: -0.097]\n\n
            
            Generate an explanation of the model prediction without mentioning LIME.\n
            Keep it short and structure it in a way that is easily comprehensible."""

instruction_no_rationale = """Your task is to explain the output of a Support Vector Machine for a user. The task is a text classification task.\n
            You will be provided with the input to the model, the model output, and confidence_score for the model output.\n\n
            
             The input is denoted as ”Input:” and is a string of text.\n
            The model prediction is denoted as ”Output:” and can be one out of three classes [politics, leisure, science].\n
            The confidence_score is denoted as ”Confidence:” and is a value between 0 and 1. A value closer to 1 indicates that the model is more confident in its prediction.\n

            Generate an explanation of the model prediction.\n
            Keep it short and structure it in a way that is easily comprehensible."""

# Generate the NLEs 

In [18]:
# Provide rationeles in prompt or not
if params['rationales']['include'] == "yes":                # Add both? 
    column_name = 'prompt'
    instruction = instruction_rationale
    new_column_name = "NLE_with_rationale"

elif params['rationales']['include'] == "no":
    column_name = 'prompt_no_rationale'
    instruction = instruction_no_rationale
    new_column_name = "NLE_no_rationale"



In [19]:

## Loop and generate NLEs  
NLEs = []
for index, row in prompts.iterrows():
    prompt = row[column_name]
    response = client.chat.completions.create(
        model=model,
        messages =[
        {"role": "system", "content": instruction},
        {"role": "user", "content": prompt}],    
          temperature = params['LLM']['temperature'],
          max_tokens = params['LLM']['max_tokens'],
          top_p = params['LLM']['top_p'],
          frequency_penalty = params['LLM']['frequency_penalty'],
          presence_penalty = params['LLM']['presence_penalty'],
          seed=42
    )  

    assistant_respone = response.choices[0].message.content

    NLEs.append(assistant_respone.strip("\n").strip())


# Add NLEs to DF 

In [20]:
# Use suiting name for column
prompts[new_column_name] = NLEs

# Save to CSV

In [22]:
prompts.to_csv('NLEs_user_study.csv', index=False)